In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.pardir)))

In [2]:
import shutil

# Set up a clean test output directory
TEST_DIR = "test"
if os.path.exists(TEST_DIR):
    shutil.rmtree(TEST_DIR)
os.makedirs(TEST_DIR, exist_ok=True)

In [3]:
import pandas as pd
import numpy as np

# Define the scenario directory
SCENARIO_PATH = os.path.join(TEST_DIR, "scenario1")
os.makedirs(SCENARIO_PATH, exist_ok=True)

# Define the results directory
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Helper function to create and display a CSV file from a DataFrame
def create_csv(df, filename):
    path = os.path.join(SCENARIO_PATH, filename)
    df.to_csv(path, index=False)

# Use otoole to initialize the scenario structure
!otoole setup csv test/scenario1/ --overwrite

In [4]:
# User Settings

# TIME
years = np.linspace(2026, 2050, num=25, dtype=int)
seasons = ['ALLSEASONS']
daytypes = ['ALLDAYS']
timebrackets = ['ALLTIMES']

# TOPOLOGY
regions = ['REGION1']

# DEMAND
demand = np.tile([100], len(years)) # MWh

# SUPPLY
technologies = ['GAS_CCGT', 'GAS_TURBINE']
residual_capacity = [0, 0] # MWh
max_capacity = [1000, 1000] # MWh
min_capacity = [0, 0] # MWh

# PERFORMANCE
operating_life = [30, 25] # years
efficiency = [0.5, 0.4] # fraction of input energy converted to output energy
capacity_factors = [0.9, 0.8] # fraction of max capacity
availability = [1.0, 1.0] # fraction of year

# ECONOMICS
discount_rate = 1e-8
capital_costs = [500, 400] # $ / MWh
fixed_costs = [0, 0] # Not implemented in PyPSA
variable_costs = [20, 25] # $ / MW

# STORAGE (NOT IMPLEMENTED IN THIS EXAMPLE)

# EMISSIONS (NOT IMPLEMENTED IN THIS EXAMPLE)

# TARGETS (NOT IMPLEMENTED IN THIS EXAMPLE)

In [5]:
# YEAR
year_df = pd.DataFrame({'VALUE': years})
create_csv(year_df, 'YEAR.csv')

# REGION
region_df = pd.DataFrame({'VALUE': regions})
create_csv(region_df, 'REGION.csv')

# TECHNOLOGY
tech_df = pd.DataFrame({'VALUE': technologies})
create_csv(tech_df, 'TECHNOLOGY.csv')

# FUEL
fuel_df = pd.DataFrame({'VALUE': ['ELEC', 'GAS']})
create_csv(fuel_df, 'FUEL.csv')

# MODE_OF_OPERATION
mode_df = pd.DataFrame({'VALUE': [1]})
create_csv(mode_df, 'MODE_OF_OPERATION.csv')

# SEASON
season_df = pd.DataFrame({'VALUE': seasons})
create_csv(season_df, 'SEASON.csv')

# DAYTYPE
daytype_df = pd.DataFrame({'VALUE': daytypes})
create_csv(daytype_df, 'DAYTYPE.csv')

# DAILYTIMEBRACKET
dailytimebracket_df = pd.DataFrame({'VALUE': timebrackets})
create_csv(dailytimebracket_df, 'DAILYTIMEBRACKET.csv')

# TIMESLICE
timeslices = [f"{s}_{d}_{t}" for s in seasons for d in daytypes for t in timebrackets]
timeslice_df = pd.DataFrame({'VALUE': timeslices})
create_csv(timeslice_df, 'TIMESLICE.csv')

In [6]:
# SpecifiedAnnualDemand
demand_df = pd.DataFrame({
    'REGION': regions * len(years),
    'FUEL': ['ELEC'] * len(years),
    'YEAR': years,
    'VALUE': demand # MWh
})
create_csv(demand_df, 'SpecifiedAnnualDemand.csv')

# SpecifiedDemandProfile
demand_profile_df = pd.DataFrame({
    'REGION': regions * len(years)*len(timeslices),
    'FUEL': ['ELEC'] * len(years)*len(timeslices),
    'TIMESLICE': np.array([timeslices] * len(years)).flatten(),
    'YEAR': np.array([years] * len(timeslices)).flatten(),
    'VALUE': [1.0/len(timeslices)] * len(years)
    # Proportion (%) of annual demand
    # CURRENTLY EVENLY DISTRIBUTED ACROSS TIMESLICES
})
create_csv(demand_profile_df, 'SpecifiedDemandProfile.csv')

# CapacityFactor
capacity_factor_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)*len(timeslices)),
    'TIMESLICE': np.array([timeslices] * len(technologies)*len(years)).flatten(),
    'YEAR': np.array([years] * len(technologies)*len(timeslices)).flatten(),
    'VALUE': np.repeat(capacity_factors, len(years)*len(timeslices))
    # Capacity factors (%) for each technology across all timeslices and years
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(capacity_factor_df, 'CapacityFactor.csv')

# CapitalCost
capital_cost_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(capital_costs, len(years))
    # Capital cost [$ per MWh]
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(capital_cost_df, 'CapitalCost.csv')

# FixedCost
fixed_cost_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(fixed_costs, len(years))
    # Fixed cost [$ per MWh per year]
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(fixed_cost_df, 'FixedCost.csv')

# CapacityToActivityUnit
capacity_to_activity_df = pd.DataFrame({
    'REGION': regions * len(technologies),
    'TECHNOLOGY': technologies,
    'VALUE': [8760] * len(technologies)
    # (MWh / capacity unit) * 8760 hours per year
    # CURRENTLY ONE VALUE PER TECHNOLOGY
    # Energy that would be produced when one unit of capacity is fully used in one year
})

# VariableCost
variable_cost_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat(variable_costs, len(years))
    # Variable cost [$ per activity unit]
    # Assume 1 activity unit = hour of operation at full capacity (i.e., 1 MW)
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(variable_cost_df, 'VariableCost.csv')

# InputActivityRatio
input_ratio_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'FUEL': np.repeat(['GAS'], len(technologies)*len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat([1.0]/np.array(efficiency), len(years))
    # MWh fuel / MWh electricity
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(input_ratio_df, 'InputActivityRatio.csv')

# OutputActivityRatio
output_ratio_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'FUEL': np.repeat(['ELEC'], len(technologies)*len(years)),
    'MODE_OF_OPERATION': [1] * len(technologies)*len(years),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': np.repeat([1.0]*len(technologies), len(years))
    # MWh electricity / MWh electricity
    # CURRENTLY ONE VALUE PER TECHNOLOGY
})
create_csv(output_ratio_df, 'OutputActivityRatio.csv')

In [7]:
# Conversionls
conv_ls_df = pd.DataFrame({
    'TIMESLICE': timeslices,
    'SEASON': np.array([seasons] * len(timeslices)).flatten(),
    'VALUE': [1 if ts.startswith(s) else 0 for ts in timeslices for s in seasons]
})
create_csv(conv_ls_df, 'Conversionls.csv')

# Conversionld
conv_ld_df = pd.DataFrame({
    'TIMESLICE': timeslices,
    'DAYTYPE': np.array([daytypes] * len(timeslices)).flatten(),
    'VALUE': [1 if f"_{daytypes[0]}_" in ts else 0 for ts in timeslices for s in daytypes]
})
create_csv(conv_ld_df, 'Conversionld.csv')

# Conversionlh
conv_lh_df = pd.DataFrame({
    'TIMESLICE': timeslices,
    'DAILYTIMEBRACKET': np.array([timebrackets] * len(timeslices)).flatten(),
    'VALUE': [1 if ts.endswith(h) else 0 for ts in timeslices for h in timebrackets]
})
create_csv(conv_lh_df, 'Conversionlh.csv')

# DaysInDayType
days_in_day_type_df = pd.DataFrame({
    'SEASON': np.array([seasons] * len(years)*len(daytypes)).flatten(),
    'DAYTYPE': np.array([daytypes] * len(years)*len(seasons)).flatten(),
    'YEAR': np.array([years] * len(daytypes)*len(seasons)).flatten(),
    'VALUE': np.array([365/len(daytypes)/len(seasons)] * len(years)*len(seasons)*len(daytypes)).flatten()
    # LEAP YEARS?
    # np.array([365] * len(years)) + (years % 4 == 0)
})
create_csv(days_in_day_type_df, 'DaysInDayType.csv')

# DaySplit
day_split_df = pd.DataFrame({
    'DAILYTIMEBRACKET': np.array([timebrackets] * len(years)).flatten(),
    'YEAR': years,
    'VALUE': np.array([1/len(timebrackets)] * len(years)).flatten()
    # Length of one timebracket in one specific day as a fraction of the year
})
create_csv(day_split_df, 'DaySplit.csv')

# YearSplit
year_split_df = pd.DataFrame({
    'TIMESLICE': np.array([timeslices] * len(years)).flatten(),
    'YEAR': years,
    'VALUE': np.array([1/len(timeslices)] * len(years)).flatten()
    # Duration of a modelled timeslice as a fraction of the year
})
create_csv(year_split_df, 'YearSplit.csv')


In [8]:
# OperationalLife
op_life_df = pd.DataFrame({
    'REGION': regions * len(technologies),
    'TECHNOLOGY': technologies,
    'VALUE': operating_life
})
create_csv(op_life_df, 'OperationalLife.csv')

# ResidualCapacity
residual_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': [0] * len(technologies)*len(years)
})
create_csv(residual_cap_df, 'ResidualCapacity.csv')

# TotalAnnualMaxCapacity
total_max_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': [1000] * len(technologies)*len(years)
})
create_csv(total_max_cap_df, 'TotalAnnualMaxCapacity.csv')

# TotalAnnualMinCapacity
total_min_cap_df = pd.DataFrame({
    'REGION': regions * len(technologies)*len(years),
    'TECHNOLOGY': np.repeat(technologies, len(years)),
    'YEAR': np.array([years] * len(technologies)).flatten(),
    'VALUE': [0] * len(technologies)*len(years)
})
create_csv(total_min_cap_df, 'TotalAnnualMinCapacity.csv')

# Discount Rate
discount = pd.DataFrame({
    'REGION': regions,
    'VALUE': [discount_rate] * len(regions)
    # CURRENTLY ONE VALUE PER REGION
})
create_csv(discount, 'DiscountRate.csv')

In [9]:
!otoole convert csv datafile test/scenario1 test/scenario1.txt ../docs/OSeMOSYS_config.yaml

In [10]:
!glpsol -m ../docs/OSeMOSYS.txt -d test/scenario1.txt --wglp test/scenario1.glp --write test/scenario1.sol

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m ../docs/OSeMOSYS.txt -d test/scenario1.txt --wglp test/scenario1.glp --write
 test/scenario1.sol
Reading model section from ../docs/OSeMOSYS.txt...
1425 lines were read
Reading data section from test/scenario1.txt...
596 lines were read
Checking Max and Min capcity-investment bounds for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 175)...
Checking Annual activity limits for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 180)...
Checking Residual and TotalAnnualMax Capacity for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 185)...
Checking Residual, Total annual maxcap and mincap investments for  all Region, Tech and Year 
Checking (line 190)...
Checking Annual production by technology bounds for r in REGION, t in TECHNOLOGY, y in YEAR 
Checking (line 195)...
Checking TimeSlices/YearSplits for y in YEAR 
Checking (line 200)...
Checking (line 201)...
Checking Model period activity

In [11]:
!otoole results glpk csv test/scenario1.sol results datafile test/scenario1.txt ../docs/OSeMOSYS_config.yaml --glpk_model test/scenario1.glp
# TODO: SOLVE WITH HIGHS

In [15]:
# 3. Create the equivalent PyPSA network
import pypsa

# Initialize the network
n = pypsa.Network()

# Set investment period
n.set_investment_periods(year_df['VALUE'])

# Add carriers
n.add("Carrier", "GAS")
n.add("Carrier", "AC") # ELEC

# Add the buses
n.add("Bus", region_df['VALUE'], carrier="AC")

# Set snapshots and weightings
snapshots = pd.Index(year_split_df[['YEAR', 'TIMESLICE']])
n.set_snapshots(snapshots)
# Weightings from YearSplit.csv
n.snapshot_weightings['objective'] = year_split_df.set_index(['YEAR', 'TIMESLICE'])
n.snapshot_weightings['generators'] = n.snapshot_weightings['objective']

demand_pypsa = pd.merge(
    demand_df,
    demand_profile_df,
    on=['REGION', 'FUEL', 'YEAR'],
    suffixes=('_annual', '_profile')
)
# Energy per snapshot = Total Annual Demand * SpecifiedDemandProfile
demand_pypsa['VALUE'] = demand_pypsa['VALUE_annual'] * demand_pypsa['VALUE_profile']
# Power per snapshot = Energy per snapshot / hours in snapshot
power = demand_pypsa\
    .pivot_table(index = ['YEAR', 'TIMESLICE'], values='VALUE')\
    .multiply(n.snapshot_weightings['objective'], axis=0)

# for region in regions:
n.add("Load",
    "demand",
    bus="REGION1",
    p_set=power['VALUE'])

# Add generators
# p_nom_extendable is True for Capacity Expansion
# marginal_cost is the fuel cost per MWh of electrical output (VariableCost / efficiency)
# capital_cost is ignored in a dispatch model, but we set it for completeness

# for region in regions:
p_max_pu = pd.DataFrame(index=snapshots) # Set time-dependent availability (CapacityFactor)
for idx, tech in enumerate(technologies):
    n.add("Generator",
          tech,
          bus="REGION1",
          carrier="GAS",
          p_nom_max=max_capacity[idx],
          p_nom_min=min_capacity[idx],
          p_nom_extendable=True,
          capital_cost=capital_costs[idx],
          marginal_cost=variable_costs[idx],
          efficiency = efficiency[idx],
          lifetime=operating_life[idx])

    p_max_pu[tech] = capacity_factors[idx]
n.generators_t.p_max_pu = p_max_pu

# Print network components to verify
print("--- PyPSA Network Components ---")
# print("\nBuses:\n", n.buses)
print("\nGenerators:\n", n.generators[['p_nom', 'p_nom_extendable', 'marginal_cost', 'efficiency']])
# print("\nLoads:\n", n.loads)
print("\nTime-varying load:\n", n.loads_t.p_set)
print("\nTime-varying generator availability:\n", n.generators_t.p_max_pu)
print("\nSnapshot Weightings:\n", n.snapshot_weightings)

INFO:pypsa.network.index:Repeating time-series for each investment period and converting snapshots to a pandas.MultiIndex.


--- PyPSA Network Components ---

Generators:
              p_nom  p_nom_extendable  marginal_cost  efficiency
name                                                           
GAS_CCGT       0.0              True           20.0         0.5
GAS_TURBINE    0.0              True           25.0         0.4

Time-varying load:
 name                              demand
2026 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2027 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2028 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2029 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2030 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2031 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2032 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2033 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2034 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2035 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2036 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2037 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2038 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2039 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2040 ALLSEASONS_ALLDAYS_ALLTIMES   100.0
2041 ALLSEASONS_ALLD

In [16]:
# 4. Run the PyPSA optimization
n.optimize()

KeyError: 'Level period not found'

In [64]:
# Print the results
print("\n--- PyPSA Optimization Results ---")
print("\nObjective:", n.objective)
print("\nOptimal Capacities (p_nom_opt):\n", n.generators.p_nom_opt)
print("\nDispatch (p):\n", n.generators_t.p)


--- PyPSA Optimization Results ---

Objective: 53500.0

Optimal Capacities (p_nom_opt):
 name
GAS_CCGT        -0.0
GAS_TURBINE    125.0
Name: p_nom_opt, dtype: float64

Dispatch (p):
 name      GAS_CCGT  GAS_TURBINE
snapshot                       
ALL_DAYS      -0.0        100.0


In [48]:
408*100/0.8 + 25*100

53500.0

In [49]:
510*100/0.9 + 20*100

58666.666666666664

In [37]:
# 5. Load and Display OSeMOSYS Results
osemosys_results_dir = 'results'

# --- Total Discounted Cost ---
total_cost_df = pd.read_csv(os.path.join(osemosys_results_dir, 'TotalDiscountedCost.csv'))
print("--- OSeMOSYS: Total Discounted Cost ---")
print(total_cost_df)
print("\n")

# --- New Capacity ---
osemosys_new_capacity_df = pd.read_csv(os.path.join(osemosys_results_dir, 'NewCapacity.csv'))
print("--- OSeMOSYS: New Capacity (p_nom_opt) ---")
print(osemosys_new_capacity_df)
print("\n")

# --- Production by Technology ---
osemosys_production_df = pd.read_csv(os.path.join(osemosys_results_dir, 'ProductionByTechnology.csv'))
print("--- OSeMOSYS: Production by Technology (Dispatch) ---")
print(osemosys_production_df)
print("\n")


--- OSeMOSYS: Total Discounted Cost ---
    REGION  YEAR        VALUE
0  REGION1  2025  4962.963203


--- OSeMOSYS: New Capacity (p_nom_opt) ---
    REGION   TECHNOLOGY  YEAR         VALUE
0  REGION1     GAS_CCGT  2025  1.111111e+02
1  REGION1  GAS_TURBINE  2025  9.844173e-15


--- OSeMOSYS: Production by Technology (Dispatch) ---
    REGION TIMESLICE   TECHNOLOGY  FUEL  YEAR         VALUE
0  REGION1  ALL_DAYS     GAS_CCGT  ELEC  2025  1.000000e+02
1  REGION1  ALL_DAYS     GAS_CCGT   GAS  2025  0.000000e+00
2  REGION1  ALL_DAYS  GAS_TURBINE  ELEC  2025  7.875338e-15
3  REGION1  ALL_DAYS  GAS_TURBINE   GAS  2025  0.000000e+00




In [17]:
# 6. Load and Display PyPSA Results

print("--- PyPSA: Objective ---")
print(n.objective)
print("\n")

print("--- PyPSA: Optimal Capacities (p_nom_opt) ---")
print(n.generators.p_nom_opt)
print("\n")

print("--- PyPSA: Dispatch (p) ---")
print(n.generators_t.p)
print("\n")


--- PyPSA: Objective ---
0.0


--- PyPSA: Optimal Capacities (p_nom_opt) ---
name
GAS_CCGT      -0.0
GAS_TURBINE   -0.0
Name: p_nom_opt, dtype: float64


--- PyPSA: Dispatch (p) ---
name      GAS_CCGT  GAS_TURBINE
snapshot                       
ALL_DAYS      -0.0         -0.0




In [18]:
# 7. Comparison Functions (Capacity Expansion)
def compare_capacity():
    # Pivot OSeMOSYS results to match PyPSA format
    osemosys_capacity= pd.read_csv(os.path.join(osemosys_results_dir, 'TotalCapacityAnnual.csv'))
    osemosys_capacity = osemosys_capacity.pivot_table(
        index='TECHNOLOGY', columns='YEAR', values='VALUE'
    )[2025]

    # Get PyPSA results
    pypsa_capacity = n.generators.p_nom_opt

    print("--- Capacity Comparison ---")
    print("\nOSeMOSYS Capacity (MWh):")
    print(osemosys_capacity)
    print("\nPyPSA Capacity (MWh):")
    print(pypsa_capacity)
    print("\n")

def compare_cost():
    """Compares total cost"""
    # Pivot OSeMOSYS results to match PyPSA format
    osemosys_cost = pd.read_csv(os.path.join(osemosys_results_dir, 'TotalDiscountedCost.csv'))
    osemosys_cost = osemosys_cost['VALUE'].sum()

    # Get PyPSA results
    pypsa_cost = n.objective
    
    print("--- Cost Comparison ---")
    print("\nOSeMOSYS Total Cost ($):")
    print(osemosys_cost)
    print("\nPyPSA Total Cost ($):")
    print(pypsa_cost)
    print("\n")

def compare_production():
    """Compares the production dispatch for each technology and timeslice"""
    # Pivot OSeMOSYS results to match PyPSA format
    osemosys_production = pd.read_csv(os.path.join(osemosys_results_dir, 'ProductionByTechnology.csv'))
    osemosys_production = osemosys_production[osemosys_production['FUEL'] == 'ELEC'].pivot_table(
        index='TIMESLICE', columns='TECHNOLOGY', values='VALUE'
    )
    
    # Get PyPSA results
    # n.generators_t.p is in MW
    # multiply by hours in each snapshot to get MWh
    pypsa_production = n.generators_t.p.multiply(n.snapshot_weightings['objective'], axis=0)

    print("--- Dispatch Comparison ---")
    print("\nOSeMOSYS Production (MWh):")
    print(osemosys_production)
    print("\nPyPSA Production (MWh):")
    print(pypsa_production)
    print("\n")

# Run comparison functions
compare_capacity()
compare_cost()
compare_production()


--- Capacity Comparison ---

OSeMOSYS Capacity (MWh):
TECHNOLOGY
GAS_CCGT       1.111111e+02
GAS_TURBINE    9.844173e-15
Name: 2025, dtype: float64

PyPSA Capacity (MWh):
name
GAS_CCGT      -0.0
GAS_TURBINE   -0.0
Name: p_nom_opt, dtype: float64


--- Cost Comparison ---

OSeMOSYS Total Cost ($):
4962.96320250217

PyPSA Total Cost ($):
0.0


--- Dispatch Comparison ---

OSeMOSYS Production (MWh):
TECHNOLOGY  GAS_CCGT   GAS_TURBINE
TIMESLICE                         
ALL_DAYS       100.0  7.875338e-15

PyPSA Production (MWh):
name      GAS_CCGT  GAS_TURBINE
snapshot                       
ALL_DAYS       NaN          NaN




In [ ]:
# HOW CAPACITY IS DETERMINED IN OSeMOSYS
# 1. RateOfDemand = MWh of demand in a timeslice if it lasted all year
#                 = 60 / (2/24) = 720 MWh for PEAK
#                 = 40 / (22/24) = 43.64 MWh for OFFPEAK
# 2. RequiredCapacity = MAX(RateOfDemand / CapacityFactor) across timeslices
#                     = MAX(43.64 / 0.9, 720 / 0.9)
#                     = 800 MWh for CCGT

# HOW DISPATCH IS INFERRED FROM OSeMOSYS
# 3. RateOfActivity / hours in the year = Power in MW

# HOW TOTAL COST IS DETERMINED IN OSeMOSYS
# 4. RequiredCapacity * CapitalCost
# 5. RequiredCapacity * FixedCost
# 6. RateOfActivity * VariableCost
# 7. SalvageValue (calculated based on OperationalLife and annuity)
# 8. TotalCost = 4 + 5 + 6 - 7